In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64
from tkinter import *

# Import the CRUD module
from CRUD import AnimalShelter

# Authenticate to the AAC database
username = "aacuser"
password = "userPassword1"
shelter = AnimalShelter(username, password)

# Retrieve all data from the database 
df = pd.DataFrame.from_records(shelter.read_all({}))

app = JupyterDash('Example')

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Position logo, dashboard title, and unique identifier
app.layout = html.Div([
    html.Center(html.B(html.H1('Search and Rescue Animal Database'))),
    html.H6("Alison Croteau"),
    html.Hr(),
    html.Div(
    # Add radio item for rescue filter options
        dcc.RadioItems(
            id="filter-type",
            options=[
                {'label': 'Dog', 'value': 'Dog'},
                {'label': 'Cat', 'value': 'Cat'}
            ],
            value='Reset',
            labelStyle=[{'display': 'inline-block'}])
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ], 
        style_table=[{'overflowX':'scroll'}],
        data=df.to_dict('records'),
        # Set up the features for the interactive data table to make it user-friendly for the client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable='multi',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,   
    ),
    html.Br(),
    html.Hr()
])
    
# Create node class and initialize the node
class Node:
    def __init__(self, data):
        self.item = data
        self.next = None
        self.prev = None
    # Create doubly linked list class
    def __init__(self):
        self.start_node = None
    # Insert Element to Empty list
    def InsertToEmptyList(self, data):
        if self.start_node is None:
            new_node = Node(data)
            self.start_node = new_node
        else:
            print("The linked list is not empty.")
    # Insert element at the end
    def InsertToEnd(self, data):
        # Check if the list is empty
        if self.start_node is None:
            new_node = Node(data)
            self.start_node = new_node
            return
        n = self.start_node
        # Iterate till the next reaches NULL
        while n.next is not None:
            n = n.next
        new_node = Node(data)
        n.next = new_node
        new_node.prev = n    
    # Create a 'next' function
    def nextNode(self, value):
        # Define current node and create results list
        currentNode = self.head
        results = []
        # Set currentNode to the next node value
        currentNode = currentNode.next
        # Add currentNode to the results list
        while currentNode is not None:
            if currentNode.has_value(value):
                results.append(currentNode)
        # Output next node
        return results
    
    # Create a 'previous' function
    def prevNode(self, value):
        # Define current node and create results list
        currentNode = self.head
        results = []
        # Set currentNode to the prev node value
        currentNode = currentNode.prev
        # Add currentNode to the results list
        while currentNode is not None:
            if currentNode.has_value(value):
                results.append(currentNode)
        # Output prev node
        return results

# Create a variable for the tkinter function
master = Tk()

#Add in callback to output selection of data
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def dashboard_results(filter_type):
    """Filter rescue dogs based on the rescue type."""
    if filter_type == 'Dog':
        df = pd.DataFrame.from_records(shelter.filtered_rescue_dogs("Dog"))
        # Insert the resulting elements into the linked list
        df.InsertToEnd()
        # Assign node next and previous values to iterations of the results
        df.next = n.next
        df.prev = n.prev
        # Create next and previous buttons
        nextButton = Button(master, text = 'Next', command = Node.nextNode)
        button.pack()
        mainloop()
        prevButton = Button(master, text = 'Previous', command = Node.prevNode)
        button.pack()
        mainloop()
        
    elif filter_type == 'Cat':
        df = pd.DataFrame.from_records(shelter.filtered_rescue_dogs("Cat"))
        # Insert the resulting elements into the linked list
        df.InsertToEnd()
        # Assign node next and previous values to iterations of the results
        df.next = n.next
        df.prev = n.prev
        # Create next and previous buttons
        nextButton = Button(master, text = 'Next', command = Node.nextNode)
        button.pack()
        mainloop()
        prevButton = Button(master, text = 'Previous', command = Node.prevNode)
        button.pack()
        mainloop()
                
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns[1:]]
    data=df.to_dict('records')
            
    return (data,columns)

app